In [1]:
import boto3
import configparser
import pandas as pd
import boto3

## Load DWH Params from a file

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("CLUSTER","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("CLUSTER","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("CLUSTER","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","DWH_CLUSTER_TYPE")
DWH_DB                 = config.get("CLUSTER","DWH_DB")
DWH_DB_USER            = config.get("CLUSTER","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DWH_DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("CLUSTER", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,multi-node
4,DWH_DB,dev
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,PassW0rd!
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,myRedshiftRole


## Create clients for IAM, EC2, S3 and Redshift
**Note**: We are creating these resources in the the **us-west-2** region. Choose the same region in the your AWS web console to the see these resources.

In [3]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="us-east-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )
s3 = boto3.resource('s3',
                       region_name="us-east-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )
iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-east-2'
                  )
redshift = boto3.client('redshift',
                       region_name="us-east-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Check out the udacity-dend data sources on S3

In [4]:
s3bucket =  s3.Bucket("udacity-dend")
print(s3bucket)

s3.Bucket(name='udacity-dend')


## Get the IAM Role

In [5]:
from botocore.exceptions import ClientError

print("Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)

Get the IAM role ARN
arn:aws:iam::277956192689:role/myRedshiftRole


## Create cluster

In [6]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists


## *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [6]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,multi-node
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,dev
5,Endpoint,"{'Address': 'multi-node.clhzwutq0ydy.us-east-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-0c7f1bcd00412d3fc
7,NumberOfNodes,4


<h2> Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

In [7]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  multi-node.clhzwutq0ydy.us-east-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::277956192689:role/myRedshiftRole


In [10]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-0e53ea96ac9ece6d5')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


## Update param in config file

In [8]:
config.set("CLUSTER","HOST", DWH_ENDPOINT)
#Write changes back to file
with open('dwh.cfg', 'w') as conf:
    config.write(conf)

In [8]:
%run create_tables.py

DROP TABLE IF EXISTS staging_events; 
DROP TABLE IF EXISTS staging_songs; 
DROP TABLE IF EXISTS songplay; 
DROP TABLE IF EXISTS users; 
DROP TABLE IF EXISTS song; 
DROP TABLE IF EXISTS artist; 
DROP TABLE IF EXISTS time; 


In [9]:
%run etl.py


    copy staging_events from 's3://udacity-dend/log_data' 
    iam_role 'arn:aws:iam::277956192689:role/myRedshiftRole'
    region       'us-west-2'
    format       as JSON 's3://udacity-dend/log_json_path.json'


    copy staging_songs from 's3://udacity-dend/song_data' 
    iam_role 'arn:aws:iam::277956192689:role/myRedshiftRole'
    region       'us-west-2'
    format       as JSON 'auto'


    INSERT INTO songplay (start_time, user_id , level , song_id , artist_id , session_id , location , user_agent)
    SELECT TIMESTAMP 'epoch' + (se.ts/1000 * INTERVAL '1 second') as start_time,
           se.userId                                              as user_id,
           se.level                                               as level,
           ss.song_id                                             as song_id,
           ss.artist_id                                           as artist_id,
           se.sessionId                                           as session_id,
           se

In [10]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'multi-node',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'awsuser',
  'DBName': 'dev',
  'Endpoint': {'Address': 'multi-node.clhzwutq0ydy.us-east-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2022, 8, 3, 12, 35, 52, 993000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0e53ea96ac9ece6d5',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-0c7f1bcd00412d3fc',
  'AvailabilityZone': 'us-east-2c',
  'PreferredMaintenanceWindow': 'sat:07:30-sat:08:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouti

In [11]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,multi-node
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,awsuser
4,DBName,dev
5,Endpoint,"{'Address': 'multi-node.clhzwutq0ydy.us-east-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-0c7f1bcd00412d3fc
7,NumberOfNodes,4


In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!